# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></div><div class="lev1 toc-item"><a href="#Load-and-Explore-Flights-Data-Years-2000-2008" data-toc-modified-id="Load-and-Explore-Flights-Data-Years-2000-2008-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load and Explore Flights Data Years 2000-2008</a></div><div class="lev1 toc-item"><a href="#What-is-the-ratio-of-Carrier-Delay-to-Number-of-flights-over-time?" data-toc-modified-id="What-is-the-ratio-of-Carrier-Delay-to-Number-of-flights-over-time?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>What is the ratio of Carrier Delay to Number of flights over time?</a></div><div class="lev1 toc-item"><a href="#Merge-in-Carrier-name-informaiton" data-toc-modified-id="Merge-in-Carrier-name-informaiton-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merge in Carrier name informaiton</a></div><div class="lev1 toc-item"><a href="#Create-Average-Carrier-Delay-Metric" data-toc-modified-id="Create-Average-Carrier-Delay-Metric-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create Average Carrier Delay Metric</a></div><div class="lev1 toc-item"><a href="#Linear-Regression-Line" data-toc-modified-id="Linear-Regression-Line-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Linear Regression Line</a></div>

# Setup

In [1]:
import glob
import pandas as pd
pd.options.display.max_columns = 300 

# Load and Explore Flights Data Years 2000-2008

In [2]:
Files = glob.glob('RawData/*')
Files

['RawData\\2003.csv',
 'RawData\\2004.csv',
 'RawData\\2005.csv',
 'RawData\\2006.csv',
 'RawData\\2007.csv']

In [3]:
pd.read_csv(Files[0],nrows=4)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2003,1,29,3,1651,1655,1912,1913,UA,1017,N202UA,141,138,119,-1,-4,ORD,MSY,837,5,17,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2003,1,30,4,1654,1655,1910,1913,UA,1017,N311UA,136,138,108,-3,-1,ORD,MSY,837,2,26,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2003,1,31,5,1724,1655,1936,1913,UA,1017,N317UA,132,138,110,23,29,ORD,MSY,837,5,17,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2003,1,1,3,1033,1035,1625,1634,UA,1018,N409UA,232,239,215,-9,-2,OAK,ORD,1835,6,11,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [4]:
ColsToRead = ['Year','Month','UniqueCarrier','TailNum','ArrDelay','DepDelay','Origin',
              'Dest','Cancelled','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
              'LateAircraftDelay']
Flights = pd.DataFrame()
for f in Files:
    Flights = Flights.append(pd.read_csv(f,usecols=ColsToRead))

In [5]:
Flights.shape

(35353543, 14)

In [6]:
Flights.sample(2)

,Year,Month,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2692952,2003,6,UA,N953UA,-16.0,-7.0,ORD,MSP,0,0.0,0.0,0.0,0.0,0.0
4558771,2007,8,OO,N953SW,-4.0,-6.0,DEN,RFD,0,0.0,0.0,0.0,0.0,0.0


In [7]:
Flights.describe()

,Year,Month,ArrDelay,DepDelay,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,3.535354e+07,3.535354e+07,3.463535e+07,3.470790e+07,3.535354e+07,3.268080e+07,3.268080e+07,3.268080e+07,3.268080e+07,3.268080e+07
mean,2.005055e+03,6.526763e+00,7.321709e+00,8.745668e+00,1.826233e-02,3.118252e+00,6.833338e-01,3.494179e+00,2.445485e-02,4.008999e+00
std,1.406436e+00,3.429465e+00,3.506306e+01,3.175675e+01,1.338985e-01,1.818229e+01,8.669848e+00,1.524667e+01,1.154963e+00,1.852936e+01
min,2.003000e+03,1.000000e+00,-1.302000e+03,-1.410000e+03,0.000000e+00,0.000000e+00,0.000000e+00,-6.000000e+01,0.000000e+00,0.000000e+00
25%,2.004000e+03,4.000000e+00,-9.000000e+00,-4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.005000e+03,7.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.006000e+03,9.000000e+00,1.100000e+01,7.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.007000e+03,1.200000e+01,2.598000e+03,2.601000e+03,1.000000e+00,2.580000e+03,1.510000e+03,1.392000e+03,5.330000e+02,1.407000e+03


# What is the ratio of Carrier Delay to Number of flights over time?

In [ ]:
# I want to only look at flights that were not cancelled
print (Flights.shape)
Flights = Flights[Flights.Cancelled==0]
print (Flights.shape)

# Merge in Carrier name informaiton

In [ ]:
Flights.UniqueCarrier.unique()

from http://aspmhelp.faa.gov/index.php/ASQP:_Carrier_Codes_and_Names
CarrierCodeToName = pd.read_clipboard()
CarrierCodeToName.to_csv('CarrierCodeToName.csv',index=False)

In [ ]:
CarrierCodeToName = pd.read_csv('CarrierCodeToName.csv')

In [ ]:
Flights = Flights.merge(CarrierCodeToName[['IATA Carrier Code','Carrier Name']], 
              how='left',left_on='UniqueCarrier',right_on='IATA Carrier Code')

In [ ]:
Flights.sample(2)

# Create Average Carrier Delay Metric

In [ ]:
#Going to ignore these Carriers as they are very small based on google research. 
Flights[pd.isnull(Flights['Carrier Name'])].UniqueCarrier.unique()

In [ ]:
print(Flights.shape)
Flights = Flights[~pd.isnull(Flights['Carrier Name'])]
print(Flights.shape)

In [ ]:
#Get needed columns
colstouse = ['Year','Month','CarrierDelay','Carrier Name']
Flights = Flights[colstouse]

#Create datetime column
Flights.rename(columns={"Year": "year", "Month": "month"},
              inplace=True)

Flights['day'] = 1

Flights['Date'] = pd.to_datetime(Flights[['year', 'month', 'day']])

In [ ]:
colstouse = ['Date','CarrierDelay','Carrier Name']
Flights = Flights[colstouse]

In [ ]:
#Ensure no NAN data 
Flights.CarrierDelay.fillna(0,inplace=True)

In [ ]:
#No Data in first half year of 2003 so need to filter out
Flights.groupby(['Date']).mean()

In [ ]:
#Filter blank dates
Flights = Flights[Flights['Date'] >= '2003-06-01']

In [ ]:
Flights.sample()

In [ ]:
#Aggregate data and get mean and counts for CarrierDelay
FlightsAgg = Flights.groupby(['Carrier Name','Date']).agg(['mean', 'count'])

In [ ]:
FlightsAgg.columns = ['mean', 'count']

In [ ]:
FlightsAgg.sample()

# Linear Regression Line

In [ ]:
from sklearn import linear_model

In [ ]:
tempDF=Flights.groupby(['Date']).agg(['mean'])

In [ ]:
tempDF.columns = ['mean']

In [ ]:
tempDF.sample(5)

In [ ]:
tempDF['DPeriod'] = tempDF.index.factorize()[0].reshape(-1,1)

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(tempDF[['DPeriod']], tempDF['mean'].values)

In [ ]:
reg.coef_

In [ ]:
tempDF['Regression'] = reg.predict(tempDF.DPeriod.values.reshape(-1, 1))

In [ ]:
tempDF=tempDF[['Regression']]

In [ ]:
tempDF['Carrier Name'] = 'Regression Best Fit'
#tempDF['count'] = 0
tempDF.rename(columns={"Regression": "mean"},
              inplace=True)

In [ ]:
tempDF = tempDF.groupby(['Carrier Name','Date']).agg(['mean', 'count'])
tempDF.columns = ['mean', 'count']

In [ ]:
FlightsFin = pd.concat([tempDF,FlightsAgg])

In [ ]:
FlightsFin.rename(columns={"mean": "Average Carrier Delay",
                          "count":"Flight Count"},
              inplace=True)

In [ ]:
#Output Data to be Visualized in Dimple
FlightsFin.to_csv("FlightData.csv")